In [102]:
import instructor
from groq import Groq
from pydantic import BaseModel
import os
from dotenv import load_dotenv
from typing import Dict, Tuple, List
import requests
from litellm import completion
import psycopg2.pool as pool
import json


In [103]:
# loading env
load_dotenv()
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
nessie_key = os.environ.get("NESSIE_API_KEY")
connection_string = os.getenv('DATABASE_URL')
open_ai_key = os.environ.get("OPEN_AI_KEY")
os.environ["OPENAI_API_KEY"] = open_ai_key

In [104]:
#data types

class UserGeneration(BaseModel):
    first_name: str
    last_name: str
    street_number:str
    street_name:str
    city:str
    state:str
    zip:str
class DataGeneration(BaseModel):
    income: float
    credit: float
    stocks: List[str]
    purchases: List[str]
    comments: str
    feedback: str
    experience: int 
    

In [105]:
#Finance data gen prompt
prompt = """I need to generate financial data. I need you to generate annual income which will be a value between 40k-300k(can be a decimal too), I want you to generate a number for credit score, I need you to generate a list of stocks owned, I need you to generate some extra comments after a user's experience talking to an AI agent which is asking financial questions to get information, I need you to generate extra comments, I need you to generate some feedback regadring the same. I need you to generate a number between 1 and 5 for experience. Generate random data because you are a bot that is generating synthetic data, try not to repeat and Return it as a JSON object. Here is an example {'income': 127850.23,
  'credit': 720.0,
  'stocks': ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'FB'],
  'income': 127850.23,
  'purchases': ['MSFT', 'AMZN', 'TSLA', 'AAPL'],
  'comments': 'The chatbot was quite helpful in guiding me through the financial questions. The interaction was smooth and the questions were relevant.',
  'feedback': 'Overall, the experience was positive. The AI agent was knowledgeable and the conversation was informative.',
  'experience': 4}"""


In [106]:
prompt2 = """I need to generate general info like name address and all which should be random and should not repeat. Please generate a user with the following information:
                - first_name
                - last_name
                - street_number
                - street_name
                - city
                - state
                - zip
                
                Return it as a JSON object."""

In [107]:
#client model
# client = instructor.from_groq(client, mode=instructor.Mode.TOOLS)
client = instructor.from_litellm(completion, mode =instructor.Mode.MD_JSON)



In [108]:
open_ai_key = os.environ.get("OPEN_AI_KEY")
os.environ["OPENAI_API_KEY"] = open_ai_key


In [110]:
# finance data gen
finance_data=[]
i =0
while i<150:

    response = client.chat.completions.create(
        model="groq/llama3-8b-8192",
        max_tokens=1024,
        messages=[{ "role": "user","content": prompt,}],
        response_model=DataGeneration,
    )
    temp_data ={"income":response.income,"credit":response.credit,"stocks":response.stocks,"income":response.income,
                "purchases":response.purchases,"comments":response.comments,"feedback":response.feedback,"experience":response.experience}
    print(temp_data)
    i+=1
    finance_data.append(temp_data)
    
    
    


{'income': 214973.56, 'credit': 850.0, 'stocks': ['AMZN', 'FB', 'GOOGL', 'MSFT', 'TSLA', 'ALK'], 'purchases': ['FB', 'MSFT', 'AMZN', 'GOOGL', 'TSLA', 'ALK'], 'comments': 'The AI agent was very patient and helped me understand my options clearly. The questions were relevant and well-structured.', 'feedback': 'I was quite pleased with the overall interaction. The AI agent was knowledgeable and the conversation was informative.', 'experience': 3}
{'income': 248190.59, 'credit': 840.0, 'stocks': ['IBM', 'TSLA', 'V', 'AXP', 'GM'], 'purchases': ['NVDA', 'MMM', 'MRK', 'JPM', 'PG'], 'comments': "Although the AI agent's questions were relevant, I felt it was a bit repetitive at times. The chatbot's tone was helpful and friendly, though.", 'feedback': 'I think the AI agent could improve by being more concise in its questions and providing more detailed answers.', 'experience': 3}
{'income': 234520.11, 'credit': 850.0, 'stocks': ['TSLA', 'BABA', 'V', 'GM', 'JPM'], 'purchases': ['JPM', 'BABA', 'V'

In [ ]:
finance_data

In [116]:
#user data gen
count =0
url = f"http://api.nessieisreal.com/customers?key={nessie_key}"
user_data=[]
i =0
while count<150:
    response2 = client.chat.completions.create(
        model="gpt-4o",
        max_tokens=1024,
        messages=[{ "role": "user","content": prompt2,}],
        response_model=UserGeneration,
    )
    customer_data = {
    "first_name": response2.first_name,
    "last_name": response2.last_name,
    "address": {
        "street_number": response2.street_number,
        "street_name": response2.street_name,
        "city": response2.city,
        "state": response2.state,
        "zip": response2.zip
    }
        }
    user_data.append(customer_data)
    print(customer_data)
    headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
        }

# Make the POST request
    response = requests.post(url, json=customer_data, headers=headers)

# Check if the request was successful
    if response.status_code == 201:
        print("Customer created successfully!")
        count+=1
    
        
    i+=1
    
    

{'first_name': 'Maxwell', 'last_name': 'Anderson', 'address': {'street_number': '5839', 'street_name': 'Maple Ave', 'city': 'Springfield', 'state': 'IL', 'zip': '62704'}}
Customer created successfully!
{'first_name': 'Emily', 'last_name': 'Smith', 'address': {'street_number': '742', 'street_name': 'Evergreen Terrace', 'city': 'Springfield', 'state': 'IL', 'zip': '62704'}}
Customer created successfully!
{'first_name': 'Alice', 'last_name': 'Johnson', 'address': {'street_number': '123', 'street_name': 'Maple Avenue', 'city': 'Springfield', 'state': 'IL', 'zip': '62704'}}
Customer created successfully!
{'first_name': 'Emily', 'last_name': 'Johnson', 'address': {'street_number': '1458', 'street_name': 'Maple Avenue', 'city': 'Springfield', 'state': 'IL', 'zip': '62704'}}
Customer created successfully!
{'first_name': 'John', 'last_name': 'Doe', 'address': {'street_number': '1234', 'street_name': 'Elm Street', 'city': 'Springfield', 'state': 'IL', 'zip': '62704'}}
Customer created successful

KeyboardInterrupt: 

In [120]:
url = f"http://api.nessieisreal.com/customers?key={nessie_key}"
headers = {
    "Accept": "application/json"
}

# Make the GET request
response = requests.get(url, headers=headers)
customers = response.json()
print(len(customers))
customers = customers[:150]
print(len(customers))
# for customer in customers:
#     print(f"Customer ID: {customer['_id']}")
#     print(f"First Name: {customer['first_name']}")
#     print(f"Last Name: {customer['last_name']}")
#     print(f"Address: {customer['address']['street_number']} {customer['address']['street_name']}, {customer['address']['city']}, {customer['address']['state']} {customer['address']['zip']}")
#     print("-" * 40)  # Separator for readability




187
150


In [ ]:
finance_data

In [177]:
# implementing Postgress through NEON

In [132]:
#create connection pool and establish connection
connection_pool = pool.SimpleConnectionPool(
    1,  # Minimum number of connections in the pool
    10,  # Maximum number of connections in the pool
    connection_string
)
if connection_pool:
    print("Connection pool created successfully")
else:
    print("connection pool missing")
conn = connection_pool.getconn()
cur = conn.cursor()

Connection pool created successfully


In [123]:
create_table_query = """
CREATE TABLE Database (
    id VARCHAR(255) PRIMARY KEY,
    first_name VARCHAR(255) NOT NULL,
    last_name VARCHAR(255) NOT NULL,
    street_number VARCHAR(255) NOT NULL,
    street_name VARCHAR(255) NOT NULL,
    city VARCHAR(255) NOT NULL,
    state VARCHAR(255) NOT NULL,
    zip VARCHAR(255) NOT NULL,
    income FLOAT NOT NULL,
    credit FLOAT NOT NULL,
    stocks JSONB NOT NULL,
    purchases JSONB NOT NULL,
    comments TEXT,
    feedback TEXT,
    experience INT NOT NULL
);
"""

# Execute the SQL query
cur.execute(create_table_query)
conn.commit()



In [ ]:
finance_data
customers

In [133]:
combined_data = []
for i in range(min(len(customers), len(finance_data))):
    customer = customers[i]
    finance = finance_data[i]
    combined_data.append({
        'id': customer['_id'],
        'first_name': customer['first_name'],
        'last_name': customer['last_name'],
        'street_number': customer['address']['street_number'],
        'street_name': customer['address']['street_name'],
        'city': customer['address']['city'],
        'state': customer['address']['state'],
        'zip': customer['address']['zip'],
        'income': finance['income'],
        'credit': finance['credit'],
        'stocks': json.dumps(finance['stocks']),
        'purchases': json.dumps(finance['purchases']),
        'comments': finance['comments'],
        'feedback': finance['feedback'],
        'experience': finance['experience']
    })

# Insert the combined data into the customer_data table
for data in combined_data:
    cur.execute('''
        INSERT INTO database (id, first_name, last_name, street_number, street_name, city, state, zip, income, credit, stocks, purchases, comments, feedback, experience) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ''', (
        data['id'], data['first_name'], data['last_name'], data['street_number'], data['street_name'], data['city'], data['state'], data['zip'], data['income'], data['credit'], data['stocks'], data['purchases'], data['comments'], data['feedback'], data['experience']
    ))

# Commit the transaction
conn.commit()

In [134]:
connection_pool.putconn(conn)# returns the connection 
connection_pool.closeall()


In [218]:
#to drop a table 
drop_table_query = """
DROP TABLE IF EXISTS customer_data;
"""

# Execute the SQL query
cur.execute(drop_table_query)

# Commit the transaction
conn.commit()

# Close the cursor and return the connection to the pool
cur.close()

In [228]:
combined_data[0]

{'id': '672f00589683f20dd518b359',
 'first_name': 'O43BQY',
 'last_name': 'E5RQ5K',
 'street_number': 'P1C8W',
 'street_name': 'O3E4R2T',
 'city': 'Y9T2O4R',
 'state': 'CA',
 'zip': '84521',
 'income': 249200.02,
 'credit_score': 789.0,
 'stocks_owned': '["CSCO", "NVDA", "INTEL", "QCOM", "TXN"]',
 'annual_income': 249200.02,
 'recent_purchases': '["GOOGL", "TSLA", "MSFT", "CSCO"]',
 'extra_comments': 'The AI agent was knowledgeable about the financial topics and the questioning was relevant to my situation.',
 'feedback': 'The interaction was helpful in addressing my financial concerns. The chatbot was responsive and easy to use.',
 'experience': 3}

In [ ]:
#### random sql operations

In [80]:
cur.execute("""SELECT first_name FROM customer_data ORDER BY first_name LIMIT 5;""")

[('',), ('',), ('',), ('',), ('',)]

In [71]:
cur.execute("""SELECT column_name
FROM information_schema.columns
WHERE table_name = 'customer_data';""")
cur.fetchall()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [23]:
rename_operations = [
    ("stocks_owned", "stocks"),
    ("recent_purchases", "purchases")
]

for old_name, new_name in rename_operations:
    cur.execute("""
    ALTER TABLE customer_data
    RENAME COLUMN {old_name} TO {new_name};
    """)



SyntaxError: syntax error at or near "{"
LINE 3:     RENAME COLUMN {old_name} TO {new_name};
                          ^


In [76]:

cur.execute("""
ALTER TABLE customer_data
RENAME COLUMN extra_comments TO comments;
""")
conn.commit()